In [1]:
import uproot
print("uproot version: ", uproot.__version__)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import pickle


uproot version:  5.0.5


In [2]:
f = uproot.open("../data_files/test2_ubobj_addpmtinfo_30_events.root")

f.items()

[('nuselection;1', <ReadOnlyDirectory '/nuselection' at 0x000106153070>),
 ('nuselection/NeutrinoSelectionFilter;1',
  <TTree 'NeutrinoSelectionFilter' (921 branches) at 0x000106153c10>),
 ('nuselection/SubRun;1', <TTree 'SubRun' (3 branches) at 0x000106153520>),
 ('nuselection/H_time;1', <TH1F (version 3) at 0x00015a0bbe80>),
 ('nuselection/H_maxH;1', <TH1F (version 3) at 0x00015a0bbee0>),
 ('nuselection/H_t0_Beam;1', <TH1F (version 3) at 0x00015a0ce0b0>),
 ('nuselection/H_TimeVsPh;1', <TH2F (version 4) at 0x00015a0cf1c0>),
 ('nuselection/H_Truthtime;1', <TH1F (version 3) at 0x00015a0ce140>),
 ('nuselection/H_SimTime;1', <TH1F (version 3) at 0x00015a0e94b0>),
 ('nuselection/H_ns_time;1', <TH1F (version 3) at 0x00015a0ea5c0>),
 ('shrreco3d;1', <ReadOnlyDirectory '/shrreco3d' at 0x00015a0ea650>),
 ('shrreco3d/_energy_tree;1',
  <TTree '_energy_tree' (6 branches) at 0x00015a0ea770>),
 ('shrreco3d/_dedx_tree;1',
  <TTree '_dedx_tree' (24 branches) at 0x00015a0eb7c0>),
 ('shrreco3d/_rcshr_

In [3]:
f["wcpselection"].items()

[('H_time;1', <TH1F (version 3) at 0x00015a1878e0>),
 ('H_maxH;1', <TH1F (version 3) at 0x00015a1a0fa0>),
 ('H_t0_Beam;1', <TH1F (version 3) at 0x00015a1a20b0>),
 ('H_TimeVsPh;1', <TH2F (version 4) at 0x00015a1a31c0>),
 ('T_eval;1', <TTree 'T_eval' (55 branches) at 0x00015a1a3370>),
 ('T_pot;1', <TTree 'T_pot' (6 branches) at 0x00015a1a2140>),
 ('T_PFeval;1', <TTree 'T_PFeval' (169 branches) at 0x00015a1b43a0>),
 ('T_spacepoints;1', <TTree 'T_spacepoints' (21 branches) at 0x00015a1a3250>),
 ('T_BDTvars;1', <TTree 'T_BDTvars' (838 branches) at 0x00015a1878b0>),
 ('T_KINEvars;1', <TTree 'T_KINEvars' (18 branches) at 0x00015a23f520>)]

In [4]:
f["wcpselection"]["T_BDTvars"].items()

[('run', <TBranch 'run' at 0x00015a49c280>),
 ('subrun', <TBranch 'subrun' at 0x00015a49cac0>),
 ('event', <TBranch 'event' at 0x00015a49d270>),
 ('nuvtx_diff', <TBranch 'nuvtx_diff' at 0x00015a49da20>),
 ('showervtx_diff', <TBranch 'showervtx_diff' at 0x00015a49e1d0>),
 ('muonvtx_diff', <TBranch 'muonvtx_diff' at 0x00015a49e980>),
 ('shw_sp_flag', <TBranch 'shw_sp_flag' at 0x00015a49f130>),
 ('shw_sp_num_mip_tracks',
  <TBranch 'shw_sp_num_mip_tracks' at 0x00015a49f8e0>),
 ('shw_sp_num_muons', <TBranch 'shw_sp_num_muons' at 0x00015a4b00d0>),
 ('shw_sp_num_pions', <TBranch 'shw_sp_num_pions' at 0x00015a4b0880>),
 ('shw_sp_num_protons', <TBranch 'shw_sp_num_protons' at 0x00015a4b1030>),
 ('shw_sp_proton_length_1',
  <TBranch 'shw_sp_proton_length_1' at 0x00015a4b17e0>),
 ('shw_sp_proton_dqdx_1', <TBranch 'shw_sp_proton_dqdx_1' at 0x00015a4b1f90>),
 ('shw_sp_proton_energy_1',
  <TBranch 'shw_sp_proton_energy_1' at 0x00015a4b2740>),
 ('shw_sp_proton_length_2',
  <TBranch 'shw_sp_proton_le

In [6]:
f["wcpselection"]["T_eval"].items()

[('run', <TBranch 'run' at 0x00015a1b4910>),
 ('subrun', <TBranch 'subrun' at 0x00015a1b5150>),
 ('event', <TBranch 'event' at 0x00015a1b5900>),
 ('file_type', <TBranchElement 'file_type' at 0x00015a1b60b0>),
 ('trigger_bits', <TBranch 'trigger_bits' at 0x00015a1b69b0>),
 ('flash_found', <TBranch 'flash_found' at 0x00015a1b71c0>),
 ('flash_found_asInt', <TBranch 'flash_found_asInt' at 0x00015a1b7970>),
 ('flash_time', <TBranch 'flash_time' at 0x00015a1d0160>),
 ('flash_measPe', <TBranch 'flash_measPe' at 0x00015a1d0910>),
 ('flash_predPe', <TBranch 'flash_predPe' at 0x00015a1d10c0>),
 ('match_found', <TBranch 'match_found' at 0x00015a1d1870>),
 ('match_found_asInt', <TBranch 'match_found_asInt' at 0x00015a1d2020>),
 ('match_type', <TBranch 'match_type' at 0x00015a1d27d0>),
 ('match_isFC', <TBranch 'match_isFC' at 0x00015a1d2f80>),
 ('match_isTgm', <TBranch 'match_isTgm' at 0x00015a1d3730>),
 ('match_notFC_FV', <TBranch 'match_notFC_FV' at 0x00015a1d3ee0>),
 ('match_notFC_SP', <TBranch 

In [ ]:
f["wcpselection"]["T_BDTvars"].arrays(["WCPMTInfoPePred", "WCPMTInfoPeMeas", "WCPMTInfoClusterLength"], library="np")

{'WCPMTInfoPePred': array([array([], dtype=float64),
        array([1.98339498e-02, 1.47845443e-02, 1.71118866e-02, 3.02381392e-02,
               1.55677206e-02, 6.10572983e-03, 2.77664360e-02, 1.54031848e-01,
               2.29997621e-01, 1.13415401e-01, 4.18812183e-01, 1.10495137e-01,
               2.50439546e-01, 1.66806189e+00, 2.46952636e+00, 1.13575990e+00,
               3.86734222e+00, 0.00000000e+00, 1.87079300e+00, 8.36973815e+00,
               8.74426994e+00, 6.55458739e+00, 7.71772219e+00, 5.67043331e+00,
               6.18250304e+00, 4.03318301e+00, 2.56852610e+00, 1.87694450e+00,
               5.54330987e+00, 1.65106018e+00, 3.11947977e+00, 2.17178802e+00]),
        array([], dtype=float64),
        array([ 5.35327922,  8.40384664,  4.01768991, 18.38308589,  4.25835175,
                6.68255725, 10.47002487, 33.60818558, 40.02728726, 39.07144213,
               72.58202738, 71.42172847, 92.64947683, 21.45687397, 14.46331634,
               48.18832646, 12.17420563

In [ ]:
f = uproot.open("../data_files/SURPRISE_Test_Samples_v10_04_07_05_Run4b_hyper_unified_reco2_BNB_nu_NC_pi0_overlay_may8_reco2_hist_62280465_snapshot.root")


In [ ]:
f.items()

[('nuselection;1', <ReadOnlyDirectory '/nuselection' at 0x00016c01e9e0>),
 ('nuselection/NeutrinoSelectionFilter;36',
  <TTree 'NeutrinoSelectionFilter' (881 branches) at 0x00013c7052a0>),
 ('nuselection/NeutrinoSelectionFilter;35',
  <TTree 'NeutrinoSelectionFilter' (881 branches) at 0x00016c01caf0>),
 ('nuselection/SubRun;1', <TTree 'SubRun' (3 branches) at 0x00016c01c880>),
 ('shrreco3d;1', <ReadOnlyDirectory '/shrreco3d' at 0x00016c01ee60>),
 ('shrreco3d/_energy_tree;4',
  <TTree '_energy_tree' (6 branches) at 0x0001752c11b0>),
 ('shrreco3d/_energy_tree;3',
  <TTree '_energy_tree' (6 branches) at 0x00016cabcd30>),
 ('shrreco3d/_energy_tree;2',
  <TTree '_energy_tree' (6 branches) at 0x0001752db970>),
 ('shrreco3d/_energy_tree;1',
  <TTree '_energy_tree' (6 branches) at 0x0001752c0a30>),
 ('shrreco3d/_dedx_tree;4',
  <TTree '_dedx_tree' (24 branches) at 0x0001752d85b0>),
 ('shrreco3d/_dedx_tree;3',
  <TTree '_dedx_tree' (24 branches) at 0x0001752c09a0>),
 ('shrreco3d/_dedx_tree;2',


In [ ]:
all_df = pd.read_pickle("../intermediate_files/all_df.pkl")

all_df.head()

In [ ]:
all_df = pd.read_pickle("../intermediate_files/all_df.pkl")

all_df.head()

,wc_br4_2_ratio1_25,wc_cme_mu_length,wc_stem_dir_angle1,wc_cosmict_3_flag_dir_weak,wc_stw_1_energy,wc_numu_cc_3_max_length,wc_cosmict_6_angle,wc_tro_3_n_muon_segs,wc_lem_shower_main_length,wc_lol_3_shower_main_length,...,wc_truth_3plusg,wc_truth_Np,wc_truth_0p,wc_truth_1mu,wc_truth_0mu,wc_truth_0pi0,wc_truth_1pi0,wc_truth_multi_pi0,topological_signal_category,physics_signal_category
0,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,False,True,False,False,True,False,True,False,2gNp,NC1pi0_Np
1,0.833333,0.0,25.868401,0.0,95.479698,13.071907,0.0,1.0,15.830915,15.830915,...,False,True,False,False,True,False,True,False,2gNp,NC1pi0_Np
2,1.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,False,True,False,False,True,False,True,False,2gNp,NC1pi0_Np
3,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,...,False,True,False,False,True,False,True,False,1gNp,NC1pi0_Np
4,1.000000,0.0,28.887241,0.0,146.860413,4.873662,0.0,1.0,31.381310,31.381310,...,False,False,True,False,True,False,True,False,2g0p,NC1pi0_0p


In [ ]:
# describe the counts for topological_signal_category
all_df["topological_signal_category"].value_counts()

0g          86271
dirt        45155
2gNp        21510
2g0p        14012
1gNp         3107
2gNp1mu      2503
1g0p         2005
3plusg          1548
1g_outFV     1165
2g_outFV      514
2g0p1mu       499
1gNp1mu       448
1g0p1mu        90
Name: topological_signal_category, dtype: int64

In [ ]:
all_df.query("filetype == 'ext'")

,wc_br4_2_ratio1_25,wc_cme_mu_length,wc_stem_dir_angle1,wc_cosmict_3_flag_dir_weak,wc_stw_1_energy,wc_numu_cc_3_max_length,wc_cosmict_6_angle,wc_tro_3_n_muon_segs,wc_lem_shower_main_length,wc_lol_3_shower_main_length,...,wc_truth_3plusg,wc_truth_Np,wc_truth_0p,wc_truth_1mu,wc_truth_0mu,wc_truth_0pi0,wc_truth_1pi0,wc_truth_multi_pi0,topological_signal_category,physics_signal_category
